# Odhad závislosti mezi proměnnými pomocí Pearsonova koeficientu a $\chi^2$ testu

Tyto metody kvantifikují vztah mezi dvěma jevy. Datasety se dvěma a více proměnnými umožňují studovat vzath mezi jednotlivými dvojicemi proměnných. Nás zajímá zejména jejich korelace či nezávislost.

## Ukázkový příklad
V tomto příkladu budeme pracovat s výsledky tenisového turnaje *ATP* do roku 2012 pro tenistu Rogera Federera. Konkrétně budeme sledovat korelaci mezi esy (speciální druh podání při kterém podávající hráč zaznamená přímý bod, aniž by se soupeř dotknul míče) a poměrem vyhraných bodů.

Na začátku nahrajeme potřebné knihovny pro zpracování dat `numpy`, `pandas`, `matplotlib` a zejména knihovnu pro statistickou analýzu, kde jsou potřebné metody implementované `scipy.stats`.

In [ ]:
import numpy as np, pandas as pd
import seaborn as sns
import scipy.stats
import matplotlib.pyplot as plt

Stáhneme dataset (je možné použít i přímo URL adresu `"https://raw.githubusercontent.com/ipython-books/cookbook-2nd-data/master/federer.csv"`)

In [ ]:
# Prerekvizity: stažení dat z webu
import requests
import os
if not os.path.exists("federer.csv"):
    url = "https://raw.githubusercontent.com/ipython-books/cookbook-2nd-data/master/federer.csv"
    r = requests.get(url)
    open("federer.csv", 'wb').write(r.content)
#%pip install numpy pandas scipy

Načteme data a zkontrolujeme, co se vytvořilo

Podíváme se, co je v tabulce konkrétně za sloupce. Abychom viděli vše pěkně, uděláme trik se změnou rozměrů a zobrazení v tabulce.

In [ ]:
pd.DataFrame(np.array(df.columns).reshape(7, 10))    

Každý řádek odpovídá jednomu zápasu, sloupce obsahují statistiku hráče a protihráče. 

Hráč 1 (`player1`) je vždy sledovaný *Roger Federer*

In [ ]:
print(f"Hráč č. 1 je:")
df['player1 name'].value_counts()

Tím už víme vše potřebné o datech, které máme k dispozici. V dalším kroku se podíváme, v kolika procentech případů *R.F.* získal body v rámci hry a v kolika procentech *esem*. Můžeme vykreslit závislost `points` a `aces`.

Vytvoříme si nový `DataFrame` se sledovanými parametry. Také odstraníme řádky, kde chybí některá z hodnot (funkce `dropna`).

In [ ]:
df_bis = pd.DataFrame({
          'points': df['player1 total points won'] / df['player1 total points total'], # pomer bodu ziskanych vyhrou
          'aces': df['player1 aces'] / df['player1 total points total'], # pomer bodu ziskany esem
          'total points': df['player1 total points total'],
          'points per minute': df['player1 total points total'] / df['time']
         }).dropna()

df_bis

Nyní můžeme data zobrazit.

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12,4)) ###
ax1.set(xlim=(0,1), xlabel='% vyhranych bodu v zapase', ylim=0, ylabel='% bodu ziskanych esem') ###
ax2.set(xlim=(0,1), xlabel='% vyhranych bodu v zapase', ylabel="freq") ###
ax3.set(xlim=(0,1), xlabel='% bodu ziskanych esem', ylabel="freq") ###


Pokud jsou dvě proměnné nezávislé, neměli bychom vidět žádný trend v mračnu bodů. 

Vizuální inspekce je však pro složitější struktury časově velmi náročná. Pro určení míry závislosti raději využijeme korelační koeficient.

Vypočteme **Pearsonův korelační koeficient** mezi poměrem výtězných bodů a poměrem bodů z es.
$$r_{xy} =\frac{\sum ^n _{i=1}(x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum ^n _{i=1}(x_i - \bar{x})^2} \sqrt{\sum ^n _{i=1}(y_i - \bar{y})^2}}$$

In [ ]:
X, Y = df_bis["points"], df_bis["aces"] ###


Výpočet je možné řešit přímo pomocí `pandas.corr()` či `scipy.stats.pearsonr()`

Hodnota 0.255 ukazuje na pozitivní korelaci mezi zkoumanými proměnnými, tzn. čím více hráč uhraje es, tím více bodů vyhraje (což se dalo očekávat). Dá se říct, že $pvalue \le 0.05$, a proto je korelace významná.

## $\chi^2$ test
Nyní určíme, zda je korelace mezi proměnnými *statisticky významná* jiným způsobem. Využijeme **$\chi^2$ chí-kvadrát testu** (chi-square test). Na začátku musíme *binarizovat* proměnné, tj. určit, zda-li je získaná hodnota větší než je ve všech turnajích obvyklé (medián).

In [ ]:
# vytvorit sloupce results, kde je points > median
# a sloupec manyaces > median


# Tvorba kontingenční tabulky
Vytvoříme si kontingenční tabulku pro všechny čtyři kombinace vstupů. 
Můžeme použít vlastní (pomalou) implementaci.

In [ ]:
# manuální tvorba kontingenční tabulky ...
import itertools
for ex_res, ex_aces in itertools.product([False, True], repeat=2):
    print(f"results={ex_res}, manyaces={ex_aces}", 
          df_bis[(df_bis["result"] == ex_res) & (df_bis["manyaces"] == ex_aces)]["result"].count())

Kontingenční tabulka je však již implementovaná v knihovně `pandas` ve funkci `crosstab`.

In [ ]:
ct ###

# Manuální výpočet $\chi^2$ testu
Pracujeme s tabulku 2x2 pro kterou vypočítáme již zmiňovaný chí-kvadrát test. Prvně musíme získat číslo $\chi^2$, které se spočítá jako
$$\chi^2=\sum_{\forall i}\frac{(O_i-E_i)^2}{E_i},$$

kde $O_i$ je získaná hodnota, $E_i$ je očekávaná hodnota: 
$$E_i=\frac{\sum_{r \in row(i)}{O_r} \cdot \sum_{c \in col(i)}{O_c}}{\sum_{\forall j}{O_j}}$$

Počítáme se stupněm volnosti $DF=(r-1)*(c-1)=1$, a korekcí 0.5 od absolutní hodnoty rozdílu $|O_i - E_i|$.

In [ ]:
# coz lze efektivne vypocitat pres numpy
values = ct.values
E = values.sum(axis=0).reshape(1, -1) * values.sum(axis=1).reshape(-1, 1) / values.sum()
xi2 = np.sum(((np.abs(values - E) - 0.5)**2) / E)
display(f"xi2={xi2}")

Nyní potřebuje spočítat pravděpodobnost $P(\chi^2 > 27.81)$. Tu lze určit na základě znalosti funkce hustoty rozdělení pravděpodobnosti, která je definována následovně
$$f(x, DF)=\frac{1}{2^{DF/2-1} \Gamma \left( DF/2 \right)} x^{DF-1} e^{ -x^2/2}.$$

Funkce je implementovaná v `scipy.stats.chi2` a hodnotu *p-value* tímpádem lze pro dané DF vypočítat jako:

# Pohodlnější přístup: knihovní funkce
Výpočet můžeme celý nechat na knihovních funkcích v scipy, konkrétně funkci [scipy.stats.chi2_contingency()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html). Tato funkce spočítá *chí-kvadrát*  statistiku a *p-hodnotu* pro test nezávislosti v kontingenční tabulce. Tato funkce vrací více hodnot (xi2, p-value, DF, E), nás zajímá druhá hodnota (p-hodnota).

Hodnota $1.33\cdot10^{-7}$ je mnohem menší než $0.05$. Proto __zamítneme nulovou hypotézu__ a můžeme prohlásit, že _je zde statisticky významná korelace mezi poměrem es vůči celkovému počtu bodů a poměrem vyhraných bodů vůči celkovému počtu bodů v zápase_.

# Závěr 

**Pozor:** [Korelace neimplikuje kauzalitu](https://cs.wikipedia.org/wiki/Korelace_neimplikuje_kauzalitu). Pokud nějaké dvě proměnné spolu korelují, nelze z toho ještě vyvozovat, že jedna je příčinou a druhá kauzálním následkem.

Cvičení inspirováno [IPython Books](https://ipython-books.github.io/74-estimating-the-correlation-between-two-variables-with-a-contingency-table-and-a-chi-squared-test/)